## 1. Tahap Preprocessing Data Teks (ingredients)

Tahap ini krusial, teman-tman. Kolom `ingredients` kita masih berupa **string yang berisi list JSON** (misalnya `["chicken","breading","oil"]`).

**Goal-nya:** Kita harus membersihkan data itu dan mengubahnya jadi format yang siap dilatih model.

---

### A. Instalasi & Import Library

**(Ini harus di-run pertama kali jika belum di-install di *environment* penelitian)**

In [7]:
# Install libraries jika belum ada
# !pip install scikit-learn numpy tqdm -y
# !pip install pillow -y 

# =======================================================
# Library Wajib untuk Training & Preprocessing
# =======================================================
import pandas as pd
import numpy as np
import re
import ast # Untuk mengonversi string list menjadi list Python

# Sklearn untuk Multi-label Encoding
from sklearn.preprocessing import MultiLabelBinarizer

# Pillow untuk Image Processing (akan digunakan nanti)
from PIL import Image

# Tqdm untuk progress bar saat pemrosesan data
from tqdm.notebook import tqdm

In [8]:
tqdm.pandas() 

# Definisikan path file CSV kamu
file_path = 'dataset/MM-Food-100k.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Data berhasil dimuat. Total {len(df)} baris.")
except Exception as e:
    print(f"ERROR: Gagal memuat data: {e}")

Data berhasil dimuat. Total 100000 baris.


### B. Cleaning dan Binarization (Label Encoding)

Ini adalah **inti dari *preprocessing* label**.

Kita akan mengonversi kolom `ingredients` menjadi **matriks biner** (nilai **0** dan **1**) yang merepresentasikan kehadiran setiap bahan unik dalam resep.

In [9]:
# 1. Konversi String List menjadi List Python
# Kolom 'ingredients' adalah string, perlu diubah jadi list of string
def convert_to_list(ingredients_str):
    try:
        # ast.literal_eval lebih aman daripada eval()
        return ast.literal_eval(ingredients_str)
    except:
        # Jika ada error konversi, kembalikan list kosong
        return []

df['ingredients_list'] = df['ingredients'].apply(convert_to_list)

# 2. Normalisasi Teks Ingredients (Opsional tapi Good Practice)
# Menghilangkan spasi ekstra, membuat lowercase, dsb.
def normalize_ingredients(ing_list):
    return [ing.strip().lower() for ing in ing_list]

df['ingredients_clean'] = df['ingredients_list'].apply(normalize_ingredients)

# 3. Encoding Multi-Label
# MultiLabelBinarizer akan menemukan semua bahan unik dan membuat kolom biner
mlb = MultiLabelBinarizer()

# Latih encoder pada semua list bahan
y_encoded = mlb.fit_transform(df['ingredients_clean'])

# Buat DataFrame dari hasil encoding
df_labels = pd.DataFrame(y_encoded, columns=mlb.classes_)

# Gabungkan dengan DataFrame utama (opsional, tapi bagus untuk inspeksi)
# df = pd.concat([df, df_labels], axis=1)

print("\n--- Hasil Encoding Ingredients ---")
print(f"Total bahan unik (output classes): {len(mlb.classes_)}")
print(f"Shape Matriks Label (y): {y_encoded.shape}")

# Tampilkan 5 bahan unik teratas
print(f"5 Bahan Unik Teratas: {mlb.classes_[:5]}")


--- Hasil Encoding Ingredients ---
Total bahan unik (output classes): 3577
Shape Matriks Label (y): (100000, 3577)
5 Bahan Unik Teratas: ['a2 milk' 'a5 wagyu beef' 'abalone' 'acai' 'accompaniments']


## 2. Tahap Preprocessing Gambar (Image Pipeline)

Di sini, kita perlu merancang **Custom Dataset dan Data Generator** (menggunakan TensorFlow/Keras) untuk menangani *file* gambar dan juga melakukan **augmentasi** saat proses *training*.

---

### A. Custom Generator (Image Data Augmentation & Resize)

Karena kita pakai arsitektur **CNN** kayak **EfficientNet**, kita wajib memastikan semua gambar punya **ukuran yang sama** (**resize**).

Selain itu, kita perlu melakukan **augmentasi** gambar. Tujuannya: bikin model kita jadi **lebih *robust*** dan nggak gampang *overfitting*.

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Library jangan lupa ya
# Download kalo belum

In [11]:
# Tentukan ukuran gambar yang diinginkan (standar untuk EfficientNet)
IMG_SIZE = 224
BATCH_SIZE = 32

# URL gambar di dataset kamu adalah URL web, jadi kita perlu 
# membuat generator yang bisa mengambil data dari URL.
# Untuk Google Colab/Jupyter Notebook, cara paling efisien 
# adalah dengan membuat generator yang mengunduh dan memuat gambar.

# Karena mengunduh 100k gambar secara real-time saat training SANGAT lambat,
# dalam konteks penelitian, kamu harus MENGUNDUH SEMUA GAMBAR terlebih dahulu 
# ke folder lokal (misal: 'dataset/images/').

# ASUMSI: Gambar sudah diunduh dan disimpan di folder 'dataset/images'
# Nama file gambar harus sesuai dengan indeks/ID di CSV.

# --- Split Data (Train/Test) ---
from sklearn.model_selection import train_test_split
# Kita gunakan index saja untuk split, bukan data mentah
train_df, test_df, y_train, y_test = train_test_split(
    df, y_encoded, test_size=0.2, random_state=42
)

# --- Image Data Generator dengan Augmentasi ---
# Augmentasi hanya untuk data training
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalisasi
    rotation_range=20,           # Augmentasi rotasi
    width_shift_range=0.2,       # Augmentasi pergeseran
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,        # Augmentasi flip
    fill_mode='nearest'
)

# Data testing/validasi hanya perlu normalisasi
test_datagen = ImageDataGenerator(rescale=1./255)

print("Image Data Generator berhasil dibuat.")

Image Data Generator berhasil dibuat.


## 3. Tahap Modeling (EfficientNet + Multi-label Head)

Di tahap ini, kita akan merancang arsitektur model utama. Kita akan menggunakan **EfficientNet** sebagai *backbone* (untuk ekstraksi fitur gambar) dan menyambungkannya dengan **Multi-label Head** (untuk memprediksi banyak bahan sekaligus). Kita juga akan menyiapkan konfigurasi *training*-nya.

---

### A. Definisi Model

* **[BANTU LANJUTIN YAA CAPE NULISNYA]**

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.applications import EfficientNetB0 

# Pastikan variabel ini sudah didefinisikan di cell sebelumnya
# IMG_SIZE = 224 # Contoh, sesuaikan dengan yang kamu pakai
# NUM_CLASSES = len(mlb.classes_)

# --- Load Pre-trained EfficientNet Backbone ---

# 1. Definisikan Input Layer secara eksplisit untuk mencegah Shape Mismatch
input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

base_model = EfficientNetB0(
    weights='imagenet',         # Load bobot ImageNet
    include_top=False,          # JANGAN sertakan top layer bawaan
    input_tensor=input_tensor,  # Gunakan input layer yang sudah didefinisikan
)

# Freeze lapisan base model agar tidak dilatih ulang
base_model.trainable = False 

# --- Tambahkan Classification Head Multi-Label ---
x = base_model.output
x = GlobalAveragePooling2D()(x) # Lapisan pooling
x = Dense(512, activation='relu')(x) 
# Lapisan Output: Jumlah neuron = Jumlah kelas unik (bahan)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(x) 

# Definisikan Model dengan Input dan Output yang sudah disiapkan
model = Model(inputs=input_tensor, outputs=predictions)

# --- Compile Model ---
model.compile(
    optimizer=Adam(learning_rate=0.001),
    # LOSS WAJIB: BinaryCrossentropy untuk Multi-Label Classification
    loss='binary_crossentropy', 
    metrics=[
        BinaryAccuracy(name='accuracy'),
        # tf.keras.metrics sudah di-import sebagai Precision/Recall
        Precision(name='precision'), 
        Recall(name='recall')
        # F1 Score: Di Tensorflow 2.13+, sudah ada F1Score() bawaan. 
        # Kalau versimu lebih lama, F1 harus dihitung dari Precision/Recall
        # F1Score(name='f1_score') 
    ]
)

print("\n--- Arsitektur Model ---")
model.summary()

ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)

### B. Pelatihan Model (Training)

Ini adalah tahap *training* model, Kita akan menjalankan proses *fitting* model menggunakan *generator* data yang sudah kita definisikan sebelumnya.

In [13]:
print("\nSelanjutnya: Buat generator gambar yang bisa memuat data dari image_url ke file lokal, lalu jalankan model.fit().")


Selanjutnya: Buat generator gambar yang bisa memuat data dari image_url ke file lokal, lalu jalankan model.fit().
